# Recipe Generator Using Custom Trained Model
This notebook generates recipes based on user-provided ingredients using a custom trained model.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder


In [ ]:
# Load and prepare data
dataset = pd.read_csv(r"/Users/edrikisaian/Documents/Advanced_AI/AdvancedAI_IndividualProject/data/recipes_data.csv")
dataset['ingredients_text'] = dataset['NER'].apply(lambda x: ' '.join(eval(x)))

# Prepare features and labels
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(dataset['ingredients_text'])
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(dataset['title'])

# Train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

print("Model training complete.")


In [ ]:
def get_users_ingredients():
    users_ingredients = []
    print("Enter the ingredients you have (type 'end' to finish):")
    while True:
        user_input = input("Ingredient: ").strip().lower()
        if user_input == 'end':
            break
        if user_input:
            users_ingredients.append(user_input)
    return users_ingredients


In [ ]:
def generate_recipe(ingredients):
    # Convert ingredients to feature vector
    ingredients_text = ' '.join(ingredients)
    X_new = vectorizer.transform([ingredients_text])

    # Get top 3 recipe predictions
    proba = model.predict_proba(X_new)
    top_indices = np.argsort(proba[0])[-3:][::-1]
    recipes = label_encoder.inverse_transform(top_indices)

    # Get recipe details from dataset
    recipe_details = []
    for recipe in recipes:
        recipe_row = dataset[dataset['title'] == recipe].iloc[0]
        details = f"Recipe: {recipe}\n"
        details += f"Ingredients: {', '.join(eval(recipe_row['NER']))}\n"
        details += f"Directions: {recipe_row['directions']}\n"
        recipe_details.append(details)

    return '\n\n'.join(recipe_details)


In [ ]:
def main():
    print("Welcome to the AI Recipe Generator!")
    ingredients = get_users_ingredients()
    if not ingredients:
        print("No ingredients provided. Exiting.")
        return
    print("\nGenerating recipe suggestions...\n")
    try:
        recipe = generate_recipe(ingredients)
        print("Here are the recipe suggestions:\n")
        print(recipe)
    except Exception as e:
        print("Error generating recipe:\n")
        print(e)
        print("\nSorry, I couldn't generate a recipe. Please try again.")


In [ ]:
main()
